<a href="https://colab.research.google.com/github/rpasquini/metodos_cuantitativos/blob/main/notebooks/R/5_Modelos_de_Clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
%load_ext rpy2.ipython


In [2]:
%%R
# Install required packages if not already installed
if (!require("nnet")) install.packages("nnet")  # For multinomial logistic regression
if (!require("sf")) install.packages("sf")      # For spatial data
if (!require("dplyr")) install.packages("dplyr") # For data manipulation
if (!require("ggplot2")) install.packages("ggplot2") # For plotting
if (!require("margins")) install.packages("margins") # For marginal effects

# Load the packages
library(nnet)
library(sf)
library(dplyr)
library(ggplot2)
library(margins)


Loading required package: nnet
Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: ggplot2
Loading required package: margins
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependency ‘prediction’

trying URL 'https://cran.rstudio.com/src/contrib/prediction_0.3.18.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/margins_0.3.28.tar.gz'

The downloaded source packages are in
	‘/tmp/RtmpKk4pJT/downloaded_packages’
In addition: Warning message:
In library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE,  :
  there is no package called ‘margins’


## Estimación de un Modelo de Elección de "Ubicación Residencial" adaptado a una situación de vivir en la formalidad vs. la informalidad.

A modo de ejemplo, vamos a intentar explicar predecir la localización de los hogares en las áreas de la "ciudad formal" o en los "asentamientos informales". Vamos a utilizar un extracto de los datos de la investigación de Goytia, Heikkila, Pasquini (2020).


## Descripción de variables
* bformal es una dummy que señala con 1 a las areas "formales" y 0 a las areas "informales". Este es un criterio de delimitación espacial.  "Informal" se utiliza para el área que constituye un barrio popular  
* dconurb1, dconurb2, dconurb3 son dummies que señalan a los agrupamientos de partidos del AMBA "Conurbano 1", "Conurbano 2" y "Conurbano 3" respectivamente, según la categorización del INDEC.  Vease https://www.indec.gob.ar/dbindec/folleto_gba.pdf
* dprop_soloviv', 'dprop_ocupant', 'dinquilinos' son dummies que señalan a los que son propietarios solo de la vivienda, ocupantes e inquilinos respectivamente.
* dedu_secincomp', 'dedu_seccomp', 'dedu_ter_o_mas' son dummies que señalan la educación del jefe de hogar
* 'dconexionagua1', 'delectricidad_med', 'dconexiongas'  son dummies que señalan la presencia de conexión al agua de red, la presencia de un medidor de electricidad y la conexión al gas de red respectivamente.


El modelo que propondremos consistirá en explicar la condición de vivir en el "área formal" a partir de un conjunto de atributos del hogar, incluyendo el ingreso, el tamaño, y un conjunto de variabes indicadores del nivel educativo del jefe de hogar.

Proponemos un Modelo Logit. Recordemos que este modelo puede pensar como el modelo de regresión lineal que es transformado mediante la función logística:

$$P(F=1)_i=\frac{1}{1+e^{-Z_i}} $$

donde

$$Z_i=\beta_0+\beta_1IngresoHogar_i+\beta_2TamañoHogar_i+\beta_3DSecundarioCompleto_i+\beta_4DTerciarioCompleto_i+\epsilon_i$$


**Observación:** Una propiedad de este modelo que puede ser útil más adelante es notar que este modelo cumple que el *ratio de las chances* de vivir en el sector formal (definido en este caso como la probabilidad de F=1 sobre la probabilidad de F=0) tomado en logaritmo se asume de forma lineal:


$$\log(\frac{P(F=1)}{P(F=0)})=\beta_0+\beta_1IngresoHogar_i+\beta_2TamañoHogar_i+\beta_3DSecundarioCompleto_i+\beta_4DTerciarioCompleto_i+\epsilon_i$$

Esta última expresión es útil pues, al aparecer P(F=0) explicita que la probabilidad de pertenecer a una categoria se define siempre en relación a otra categoría. Esto será útil para la interpretación del modelo multi-categoría.


In [3]:
%%R
# Levantamos los datos
datosghp <- read.csv("https://raw.githubusercontent.com/rpasquini/urban-econometrics/master/data/formal%20premia%20data%20extract.csv")

# Mostramos los primeros registros
head(datosghp)


  hora_c hora_f bformal dconurb1 dconurb2 dconurb3 dmore2000 dprop_soloviv
1  14.30  14.55       0        0        0        0         1             1
2  13.30  13.55       0        0        0        0         1             0
3  14.25  14.50       0        0        0        0         1             1
4  11.05  11.30       0        0        0        0         1             1
5   9.35  10.00       0        0        0        0         1             1
6  10.40  11.05       0        0        0        0         1             1
  dprop_ocupant dinquilinos npersonashogar dedu_secincomp dedu_seccomp
1             0           0              4              0            0
2             0           1              4              1            0
3             0           0              4              1            0
4             0           0              4              0            1
5             0           0              8              0            0
6             0           0              3       

In [20]:
%%R
logit_model <- glm(bformal ~ ingresohogar2 + npersonashogar + dedu_seccomp + dedu_ter_o_mas,
                   data = datosghp,
                   family = binomial(link = "logit"))
summary(logit_model)


Call:
glm(formula = bformal ~ ingresohogar2 + npersonashogar + dedu_seccomp + 
    dedu_ter_o_mas, family = binomial(link = "logit"), data = datosghp)

Coefficients:
                 Estimate Std. Error z value Pr(>|z|)   
(Intercept)    -8.477e-01  2.669e-01  -3.176  0.00150 **
ingresohogar2   3.727e-05  1.374e-05   2.713  0.00667 **
npersonashogar -1.407e-02  4.709e-02  -0.299  0.76504   
dedu_seccomp    6.655e-01  2.107e-01   3.158  0.00159 **
dedu_ter_o_mas  6.764e-01  3.712e-01   1.822  0.06843 . 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 690.49  on 500  degrees of freedom
Residual deviance: 666.66  on 496  degrees of freedom
  (49 observations deleted due to missingness)
AIC: 676.66

Number of Fisher Scoring iterations: 4



## Calculamos los efectos marginales.
Vamos a usar un paquete especifico para esto, el paquete **margins**
Por default, margins va a calcular el **efecto marginal promedio**, es decir, primero computará el efecto marginal para cada observación de la muestra, y luego reportará el promedio de todas las observaciones.

In [14]:
%%R
# Calculamos los efectos marginales promedio

margins_modelo <- margins(logit_model)
summary(margins_modelo)


         factor     AME     SE       z      p   lower  upper
   dedu_seccomp  0.1573 0.0478  3.2887 0.0010  0.0635 0.2510
 dedu_ter_o_mas  0.1599 0.0866  1.8462 0.0649 -0.0099 0.3296
  ingresohogar2  0.0000 0.0000  2.7923 0.0052  0.0000 0.0000
 npersonashogar -0.0033 0.0111 -0.2990 0.7650 -0.0251 0.0185


Si quisieramos calcular el efecto marginal en el valor promedio de las variables, tenemos que pasar explícitamente en qué valores vamos calcular el efecto marginal, lo hacemos pasando una lista de valores

In [28]:
%%R
# Pasamos una lista list() que incluye los valores promedio de las variables que me interesan
# Notar que el valor promedio de las dummies no tiene mucho sentido, por lo que podrían cambiar esos valores por 0 o 1 de acuerdo a cual sea el punto de interes.
mfx_at_mean <- margins(logit_model, at = list(
  ingresohogar2 = mean(datosghp$ingresohogar2, na.rm = TRUE),
  npersonashogar = mean(datosghp$npersonashogar, na.rm = TRUE),
  dedu_seccomp = mean(datosghp$dedu_seccomp, na.rm = TRUE),
  dedu_ter_o_mas = mean(datosghp$dedu_ter_o_mas, na.rm = TRUE)
))
summary(mfx_at_mean)

         factor ingresohogar2 npersonashogar dedu_seccomp dedu_ter_o_mas
   dedu_seccomp    13339.5210         4.1727       0.2709         0.0745
 dedu_ter_o_mas    13339.5210         4.1727       0.2709         0.0745
  ingresohogar2    13339.5210         4.1727       0.2709         0.0745
 npersonashogar    13339.5210         4.1727       0.2709         0.0745
     AME     SE       z      p   lower  upper
  0.1651 0.0523  3.1569 0.0016  0.0626 0.2675
  0.1678 0.0921  1.8215 0.0685 -0.0127 0.3483
  0.0000 0.0000  2.7126 0.0067  0.0000 0.0000
 -0.0035 0.0117 -0.2989 0.7650 -0.0264 0.0194


Tambien podemos calcular en valores especificos. Por ejemplo lo calculamos para una familia con cuatro integrantes, ingreso del hogar de $14000, y jefe con secundario incompleto

In [26]:
%%R
# Marginal effects at specific values
mfx_specific <- margins(logit_model, at = list(
  ingresohogar2 = 1400,
  npersonashogar = 4,
  dedu_seccomp = 0,
  dedu_ter_o_mas = 0
))
summary(mfx_specific)

         factor ingresohogar2 npersonashogar dedu_seccomp dedu_ter_o_mas
   dedu_seccomp     1400.0000         4.0000       0.0000         0.0000
 dedu_ter_o_mas     1400.0000         4.0000       0.0000         0.0000
  ingresohogar2     1400.0000         4.0000       0.0000         0.0000
 npersonashogar     1400.0000         4.0000       0.0000         0.0000
     AME     SE       z      p   lower  upper
  0.1395 0.0428  3.2609 0.0011  0.0557 0.2233
  0.1418 0.0783  1.8102 0.0703 -0.0117 0.2953
  0.0000 0.0000  3.2184 0.0013  0.0000 0.0000
 -0.0030 0.0099 -0.2993 0.7647 -0.0223 0.0164


## Predicción

¿Cuál es la probabilidad de que una familia de que una familia con cuatro integrantes, ingreso del hogar de $14000, y jefe con secundario incompleto viva en la formalidad?


In [ ]:
%%R
# Creamos un nuevo dataframe con los valores para predecir
nuevos_datos <- data.frame(
  ingresohogar2 = 14000,
  npersonashogar = 4,
  dedu_seccomp = 0,
  dedu_ter_o_mas = 0
)


# Luego usamos el comando predict, y tenemos que pasarle el modelo ajustado, los nuevos datos, y especificar el tipo de respuesta 'response'
# es importante no olvidar 'response', ya que la respuesta default de predict es solo la parte lineal del modelo, y faltaría aplicar la función logística para llegar a la predicción

prob <- predict(logit_model, newdata = nuevos_datos, type = "response")
print(paste("Probabilidad de vivir en la formalidad:", round(prob, 4)))


# Versión Probit
El modelo Probit representa una alternativa al modelo Logit. La función que realiza la transformación en este caso es la [función acumulativa](https://en.wikipedia.org/wiki/Cumulative_distribution_function) de la [Distribución Normal Estandar](https://en.wikipedia.org/wiki/Normal_distribution), que es típicamente denotada $\Phi$. Es decir:

$$P(F=1)=\Phi(\beta_0+\beta_1IngresoHogar_i+\beta_2TamañoHogar_i+\beta_3DSecundarioCompleto_i+\beta_4DTerciarioCompleto_i+\epsilon_i)$$


La estimación del Probit es muy similar, solo tengo que especificar la familia de modelos probit: `family = binomial(link = "probit")`

In [29]:
%%R
# Ajustamos el modelo probit
modelo_probit <- glm(bformal ~ ingresohogar2 + npersonashogar + dedu_seccomp + dedu_ter_o_mas,
                       data = datosghp,
                     , family = binomial(link = "probit"))

# Calculamos los efectos marginales promedio
margins_probit <- margins(modelo_probit)
summary(margins_probit)


         factor     AME     SE       z      p   lower  upper
   dedu_seccomp  0.1584 0.0484  3.2717 0.0011  0.0635 0.2533
 dedu_ter_o_mas  0.1626 0.0868  1.8737 0.0610 -0.0075 0.3327
  ingresohogar2  0.0000 0.0000  2.7983 0.0051  0.0000 0.0000
 npersonashogar -0.0033 0.0111 -0.2940 0.7688 -0.0251 0.0185


# Estimando un Modelo de Localización Residencial en la Ciudad. Aplicación del modelo de múltiples categorías.

El modelo a estimar será un Multinomial Logit. Este modelo se puede pensar como una generalización del modelo Logit para una cantidad de K categorías. Podemos pensar a este modelo como la modelización de K-1 ecuaciones tomando como base a una categoría de referencia (la K-esima).  

Al igual que el Logit, este modelo tiene la propiedad de que el logaritmo del ratio de las chances entre las categorías se modela de manera lineal. En este caso, el logaritmo del ratio de las chances de una categroria k-esima en relación a la categoría K (la última), se puede escribir como:

$$ \ln(\frac{\Pr(Y=k)}{\Pr(Y=K)})=\beta_{0,k}+\beta_{1,k}X_i+\epsilon_i$$

El modelo tiene similares implicancias al Logit de dos categorías: Los efectos del modelo en las probabilidades serán no-lineales (i.e., dependerán de los valores de las variables explicativas) y será necesario estimar los *efectos marginales*.


In [ ]:
%%R
# Descargamos y leemos el shapefile de comunas
temp <- tempfile()
download.file("https://github.com/rpasquini/econometrics_and_causality/blob/master/comunas.zip?raw=true",
              temp)
unzip(temp)
comunas <- st_read("comunas.shp")

# Visualizamos el mapa de comunas
ggplot() +
  geom_sf(data = comunas, aes(fill = factor(COMUNAS))) +
  scale_fill_viridis_d() +
  theme_minimal() +
  labs(title = "Comunas de Buenos Aires",
       fill = "Comuna")


In [30]:
%%R
# Cargamos los datos de la EAH
dataeah <- read.csv("https://github.com/rpasquini/econometrics_and_causality/blob/master/eah2016.csv?raw=true")

# Filtramos para quedarnos solo con los jefes de hogar
dataeah <- dataeah[dataeah$parentes_2 == 1,]

# Mostramos las primeras filas
head(dataeah)


  id nhogar miembro comuna                                dominio edad  sexo
1  1      1       1     13                           resto ciudad   18 mujer
2  2      1       1      1                           resto ciudad   18 varon
3  3      1       1      6 inquilinatos hoteles inmuebles tomados   18 mujer
4  4      1       1     14                           resto ciudad   18 varon
5  5      1       1      2                           resto ciudad   18 varon
6  6      1       1     13                           resto ciudad   18 mujer
  parentes_2 p5_2 p6_a p6_b     estado categori t13 t14 t18 t28 t29 t29a t30
1          1    6   95   95    ocupado        3   0   0   0   0   0    0   2
2          1    6   95   95   inactivo        0   2   2   0   0   0    0   0
3          1    6   95   95    ocupado        3   0   0   0   0   0    0   2
4          1    6   95   95   inactivo        0   2   2   0   0   0    0   0
5          1    6   95   95   inactivo        0   2   2   0   0   0    0   0

In [31]:
%%R
# Ajustamos el modelo multinomial
modelo_mn <- multinom(comuna ~ ingtot_2 + edad, data = dataeah)

# Calculamos probabilidades predichas para un caso específico
nuevos_datos <- data.frame(
  ingtot_2 = 20000,
  edad = 60
)

# Predicción
predicciones <- predict(modelo_mn, newdata = nuevos_datos, type = "probs")
print("Probabilidades predichas por comuna:")
print(predicciones)


# weights:  60 (42 variable)
initial  value 16248.301207 
iter  10 value 16053.814200
iter  20 value 15929.267393
iter  30 value 15830.043736
iter  40 value 15746.014038
final  value 15743.371348 
converged
[1] "Probabilidades predichas por comuna:"
         1          2          3          4          5          6          7 
0.10009694 0.07522061 0.08660063 0.07661415 0.06907164 0.07482393 0.07454716 
         8          9         10         11         12         13         14 
0.05730268 0.04207195 0.04501077 0.03300695 0.06219320 0.07325591 0.06954351 
        15 
0.06063997 
